In [ ]:
import markdown
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
file = open('/content/drive/MyDrive/book-md/book.md', 'r')
text = markdown.markdown(file.read())

In [ ]:
def clean(text: str) -> str:
    """Basic text formatting."""
    formatted_text = text.replace("\n", " ").strip()
    return formatted_text

In [ ]:
text = clean(text)

In [ ]:
!pip install semantic-text-splitter==0.2.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 93.3 MB/s eta 0:00:00


In [ ]:
from semantic_text_splitter import HuggingFaceTextSplitter as hfts
from tokenizers import Tokenizer

In [ ]:
min_tokens = 100
max_tokens = 350

In [ ]:
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
splitter = hfts(tokenizer, trim_chunks=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
chunks = splitter.chunks(text, chunk_capacity=max_tokens)

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n {chunk}")

Chunk 1:
 <h1>Classes 9-10</h1> <p><img alt="0_Image_0.Png" src="0_Image_0.Png" /></p> <p>NATIONAL CURRICULUM &amp; TEXTBOOK BOARD, DHAKA</p> <p><img alt="0_image_1.png" src="0_image_1.png" /></p> <p>Prescribed by National Curriculum and Textbook Board as a Textbook for Classes Nine-Ten from the academic Year 2013 Physics [ Classes-IX-X ] Written by Dr. Shahjahan Tapan Dr. Rana Chowdhury Dr. Ekram Ali Sheikh Dr. Rama Bijoy Sarker Edited by Dr. Ali Asgar Translated by Dr. Rama Bijoy Sarker Subed Chandra Paul Md. Khairul Alam National Curriculum and Textbook Board, Dhaka Published by National Curriculum and Textbook Board 69-70, Motijheel Commercial Area, Dhaka [All rights reserved by the Publisher] Trial Edition First Publication : December 2012 Coordinated by Md. Mukhlesur Rahman Computer Compose Laser Scan Ltd.</p> <p>Cover Sudarshan Bachar Sujaul Abedeen Illustrator Md. Hasanul Kabir Sohag Design National Curriculum and Textbook Board For Free Distribution from academic year 2012 by 

In [ ]:
# Assuming chunks is a list of strings
with open('output.txt', 'w') as f:
    for chunk in chunks:
        f.write(chunk + '\n')  # Add a newline for readability


In [ ]:
page_chunks = []
for i, chunk in enumerate(chunks):
    page_chunks.append({
        "character_count": len(chunk),
        "word_count": len(chunk.split()),
        "token_count": len(chunk) // 4,
        "chunk": chunk,
    })

print(page_chunks[56])

{'character_count': 1505, 'word_count': 271, 'token_count': 376, 'chunk': 'The motion of a car in a straight street is linear motion. Rotational motion : When a body rotates around a particular point or an axis keeping the distance of the particles of the body unchanged is called rotational motion. For example, motion of an electric fan, motion of the heads of a clock.</p> <p>Rectilinear motion : When a body moves along a straight line in such a way that each particle of the body travels the same distance at the same time in the same direction. If a book is pushed to shift from one end to the other of a table without rotation, the motion will be rectilinear because all the particles of the book travel the equal distance, at equal time in the same direction.</p> <p>Periodic motion : If the motion of a moving particle is such that it passes through a definite point along the path of its motion in the same direction in a definite interval of time, this type of motion is called periodic mo

In [ ]:
import pandas as pd

df = pd.DataFrame(page_chunks)

In [ ]:
df.describe().round(2)

,character_count,word_count,token_count
count,465.00,465.00,465.00
mean,1203.70,198.26,300.53
std,288.35,51.64,72.10
min,345.00,29.00,86.00
25%,1031.00,165.00,257.00
50%,1222.00,203.00,305.00
75%,1410.00,238.00,352.00
max,2371.00,298.00,592.00


In [ ]:
df["token_count"]

,token_count
0,324
1,455
2,433
3,354
4,266
...,...
460,367
461,288
462,249
463,132


In [ ]:
import numpy as np
np.std(df["token_count"])

72.0212014493181

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer # requires !pip install sentence-transformers
from sentence_transformers import util
from tqdm.auto import tqdm

dimensions = 768
embedding_model = SentenceTransformer(model_name_or_path="mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions, device="cuda")

#test_query = "Represent this sentence for searching relevant passages: I like reading books"

#sentences = [
#    test_query,
#    "My hobby is reading",
#    "Literature is my favorite subject",
#    "The earth is flat",
#    "I hate reading"
#]

#test_embeddings = embedding_model.encode(sentences)
#similarities = util.cos_sim(test_embeddings[0], test_embeddings[1:])

#print('similarities:', similarities)

In [ ]:
import os
embeddings_path = "embeddings.csv"

In [ ]:
%%time
from tqdm.auto import tqdm # To see progress bar

embedding_model.to("cuda")

if not os.path.exists(embeddings_path):
    for page in tqdm(page_chunks):
        page["embedding"] = embedding_model.encode(page["chunk"])
    embeddings_df = pd.DataFrame(page_chunks)
    embeddings_df.to_csv(embeddings_path, index=False)

  0%|          | 0/465 [00:00<?, ?it/s]

CPU times: user 37.9 s, sys: 266 ms, total: 38.1 s
Wall time: 39.7 s


In [ ]:
# Load the embeddings from the csv file
embeddings = pd.read_csv(embeddings_path)
embeddings.head()

,character_count,word_count,token_count,chunk,embedding
0,1297,185,324,"<h1>Classes 9-10</h1> <p><img alt=""0_Image_0.P...",[-3.74677390e-01 -2.20031485e-01 -1.45054869e-...
1,1822,281,455,In order to build up a nation imbued with the ...,[ 1.98076125e-02 3.47505808e-02 -5.58695316e-...
2,1732,265,433,While selecting the contexts and their present...,[-3.71419415e-02 -5.23377895e-01 -5.69244139e-...
3,1419,174,354,Mostafa Kamaluddin Chairman National Curriculu...,[-3.72647226e-01 -4.33325410e-01 6.68514729e-...
4,1066,167,266,It is inextricably related to each and every a...,[-1.01805024e-01 3.21075231e-01 3.94026250e-...


In [ ]:
import random
import numpy as np
import torch

device = "cuda"

embeddings = pd.read_csv(embeddings_path)
embeddings["embedding"] = embeddings["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" ")) # Arrays got converted to strings in csv so we need to convert them back
page_chunks = embeddings.to_dict(orient="records")
embeddings["embedding"]

,embedding
0,"[-0.37467739, -0.220031485, -0.0145054869, -0...."
1,"[0.0198076125, 0.0347505808, -0.558695316, -0...."
2,"[-0.0371419415, -0.523377895, -0.0569244139, -..."
3,"[-0.372647226, -0.43332541, 0.668514729, -0.08..."
4,"[-0.101805024, 0.321075231, 0.39402625, -0.808..."
...,...
460,"[0.609173894, -0.351702362, 0.570304155, 0.105..."
461,"[0.4443709, 0.07081486, 0.8170414, -0.37784636..."
462,"[0.243853614, 0.190258026, 0.78078258, -0.7084..."
463,"[0.4719893, 0.19988388, 0.253406, 0.07151154, ..."


In [ ]:
embeddings = torch.tensor(np.stack(embeddings["embedding"].tolist(), axis=0), dtype=torch.float32).to("cuda") # Convert to tensors so that we can utilize the gpu for computing
embeddings

tensor([[-0.3747, -0.2200, -0.0145,  ..., -0.1602, -0.6114,  0.1060],
        [ 0.0198,  0.0348, -0.5587,  ..., -0.0303, -0.5584,  0.6496],
        [-0.0371, -0.5234, -0.0569,  ...,  0.0053, -1.1606, -0.1104],
        ...,
        [ 0.2439,  0.1903,  0.7808,  ...,  0.2563, -0.7798,  0.1103],
        [ 0.4720,  0.1999,  0.2534,  ...,  0.1575, -0.4380,  0.9148],
        [-0.8215,  0.0161,  0.1258,  ..., -0.2784, -0.7439,  0.4064]],
       device='cuda:0')

In [ ]:
len(embeddings)

465

In [ ]:
question = "Explain the math of Ovi and Mitu"
print(f"Query: {question}")

question_embedding = embedding_model.encode(question, convert_to_tensor=True).to(device)

from time import perf_counter as timer

start = timer()
_scores = util.cos_sim(a=question_embedding, b=embeddings)[0]
end = timer()

print(f"Time taken to get similarity scores on {len(embeddings)}: {end - start:.5f} seconds")

_results = torch.topk(_scores, k=2)
_results

Query: Explain the math of Ovi and Mitu
Time taken to get similarity scores on 465: 0.08783 seconds


torch.return_types.topk(
values=tensor([0.7350, 0.5752], device='cuda:0'),
indices=tensor([ 61, 171], device='cuda:0'))

In [ ]:
page_chunks[61]

{'character_count': 1300,
 'word_count': 228,
 'token_count': 325,
 'chunk': 'The dimension of displacement is the same as that of distance.</p> <p>Therefore, [ s ] = L The unit of displacement is the same as that of distance that is meter (m). The displacement of a body is 50m towards north means that the body has moved 50m from its initial position towards north direction.</p> <p>Speed : Suppose in the previous example, Ovi takes 50 seconds, to travel 100m distance. If Mitu covers the same distance in 40 seconds, who goes faster ? Ovi or Mitu, definitely Mitu goes faster because she takes less time. Suppose, Ovi travels 100m in 50 seconds, Mitu travels 75m in 30 seconds can we say Ovi goes slower than Mitu ? Does not Ovi travel more distance than Mitu ? The distance of a particular time Ovi and Mitu travels has to be compared to know who goes faster. Let the particular time be 1 second therefore,</p> <p>In 1 second Ovi travels $\\dfrac{100}{50}$ .  $${\\frac{0}{2}}=2{\\mathrm{~meter}

In [ ]:
import textwrap
# Print text in a formatted way
def format_text(text, length=80):
    passage = textwrap.fill(text, length)
    print(passage)

In [ ]:
print(f"Query: {question}\n")
print("Results: ")

for value, indices in zip(_results[0], _results[1]):
    print(f"Score: {value:.5f}")
    print("Passage: ")
    format_text(page_chunks[indices]["chunk"])
    print(f'Page Number: {page_chunks[indices]}')
    print("\n")

Query: Explain the math of Ovi and Mitu

Results: 
Score: 0.73499
Passage: 
The dimension of displacement is the same as that of distance.</p> <p>Therefore,
[ s ] = L The unit of displacement is the same as that of distance that is meter
(m). The displacement of a body is 50m towards north means that the body has
moved 50m from its initial position towards north direction.</p> <p>Speed :
Suppose in the previous example, Ovi takes 50 seconds, to travel 100m distance.
If Mitu covers the same distance in 40 seconds, who goes faster ? Ovi or Mitu,
definitely Mitu goes faster because she takes less time. Suppose, Ovi travels
100m in 50 seconds, Mitu travels 75m in 30 seconds can we say Ovi goes slower
than Mitu ? Does not Ovi travel more distance than Mitu ? The distance of a
particular time Ovi and Mitu travels has to be compared to know who goes faster.
Let the particular time be 1 second therefore,</p> <p>In 1 second Ovi travels
$\dfrac{100}{50}$ .  $${\frac{0}{2}}=2{\mathrm{~meter}}$$ I

In [ ]:
# Function to perform similarity search and get the scores.
def retrieve(query: str, embeddings: torch.tensor, embedding_model: SentenceTransformer=embedding_model, retrievables: int=2):
    _embedding = embedding_model.encode(query, convert_to_tensor=True).to("cuda")

    start = timer()
    sim_scores = util.cos_sim(_embedding, embeddings)[0]
    end = timer()

    print(f"Time taken to get scores on {len(embeddings)} embeddings: {end - start:.5f} seconds")

    values, indices = torch.topk(sim_scores, k=retrievables)
    return sim_scores, indices

# Fucntion to print the results from the retrieve function.
def print_results(query: str, embeddings: torch.tensor, embedding_model: SentenceTransformer, page_chunks: list[dict]=page_chunks, retrievables=2):
    values, indices = retrieve(query=query, embeddings=embeddings, retrievables=retrievables)
    print(f"Query: {query}\n")
    print("Results: ")

    for index in indices:
        print("Passage: ")
        format_text(page_chunks[index]["chunk"])
        print(f'Page Number: {page_chunks[index]}')
        print("\n")

In [ ]:
#def format_text_for_chat(input_text, user_name="user", model_name="assistant"):
#    # Format the text
#    formatted_text = "<|system|>\nYou are a helpful assistant.<|end|>\n"
#    formatted_text += f"<|{user_name}|>\n{input_text}\n<|end|>\n"
#    formatted_text += f"<|{model_name}|>\n"
#
#    return formatted_text

# Example usage
#input_text = "How to explain Internet for a medieval knight?"
#formatted = format_text_for_chat(input_text)
#print(formatted)


In [ ]:
def format_text_for_chat(input_text, user_name="user", model_name="embedding_model"):
    formatted_text = "<bos>\n"
    formatted_text += f"<start_of_turn>{user_name}\n"
    formatted_text += f"{input_text}\n"
    formatted_text += "<end_of_turn>\n"
    formatted_text += f"<start_of_turn>{model_name}\n"
    return formatted_text

# Example usage
input_text = "Write a hello world program"
formatted = format_text_for_chat(input_text)
print(formatted)

<bos>
<start_of_turn>user
Write a hello world program
<end_of_turn>
<start_of_turn>embedding_model



In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13320    0 13320    0     0  36835      0 --:--:-- --:--:-- --:--:-- 36795
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 7.5 MB/s eta 0:00:00


## VS Code cdi
curl https://ollama.ai/install.sh | sh
ollama serve &
ollama pull model name

In [ ]:
%xterm

Launching Xterm...

<IPython.core.display.Javascript object>

In [ ]:
# Helper function for text formatting
def format_text(text, length=80):
    return text

def format_text_for_chat(text):
    return text

# Shared state to control text generation
class GenerationState:
    def __init__(self):
        self.stop_generation = False

generation_state = GenerationState()

def prompt_formatter(query: str, context_items: list[dict]) -> str:
    context = ". " + "\n- ".join([page["chunk"] for page in context_items])
    base_prompt = r'''You are an assistive physics chatbot named Teammit who answers questions for 9th-grade students using their physics textbook. You can only answer physics-based questions. Follow these guidelines:

1. Carefully read the entire question and identify all parts that need to be answered.
2. Give yourself room to think by extracting relevant passages from the context before answering the query.
3. Present a clear and concise solution without unnecessary filler text.
4. Use a structured approach to problem-solving:
   a) **List given information**
   b) **State the correct formula(s)**, making sure signs are appropriately used
   c) **Explain why the chosen formula** is suitable for this problem
   d) **Outline solution steps** and substitute values correctly
   e) **Perform all necessary calculations** for each part of the question
   f) **State and explain the final answer** based on the calculations
5. Ensure you complete **all** calculations requested in the question. Double-check that you've addressed every part of the query.
6. **Never stop** at explanation; make sure you conclude who or what satisfies the query.
7. **Always base your conclusion on the calculations you performed**, even if it differs from examples in the textbook.
8. Use the formatting instructions provided (Markdown headers, LaTeX for equations, etc.)
9. Explain your reasoning clearly, making the solution process easy to follow.
10. If you cannot answer the question from the given context of the book, say: "Sorry, I cannot answer this question as it is not covered in your textbook."
11. **For math problems**, state the symbols for the terms, e.g., Distance = *s*, and then calculate using the symbols.
12. Make sure your answers are as explanatory as possible.
13. Before concluding, **review the original question** to ensure all parts have been answered. If any part is missing, complete the calculation and add it to your response.
14. For follow-up questions, first search for the context in your past conversations and then refer to the book.
15. When solving physics problems, always consider the following key equations and choose the **most appropriate one** based on the given information:
    - **For motion:**
      * $$s = ut + \frac{{1}}{{2}}at^2$$ (use when displacement, time, and either initial velocity or acceleration are known)
      * $$v = u + at$$ (use when initial velocity, time, and either final velocity or acceleration are known)
      * $$v^2 = u^2 + 2as$$ (use when initial and final velocities, and either displacement or acceleration are known)
    - **For force:** $$F = ma$$
    - **For work and energy:** $$W = Fs$$, $$KE = \frac{{1}}{{2}}mv^2$$, $$PE = mgh$$
    - **For momentum:** $$p = mv$$
    **Always explain why you chose a particular equation and how it applies to the given problem.**
16. If multiple formulas could potentially apply, **explain your reasoning** for choosing one over the others.
17. When dealing with acceleration problems, pay special attention to whether initial velocity is given or can be assumed to be zero. **Never assume a final velocity** unless it's explicitly stated or can be directly calculated from given information.
18. **Always list out all given information** and clearly state any assumptions made, explaining why those assumptions are reasonable.

**IMPORTANT FORMATTING INSTRUCTIONS:**
1. Use Markdown headers for section titles (e.g., # Main Title, ## Subtitle, ### Subsubtitle).
2. For mathematical equations displayed on their own line (block equations), use LaTeX syntax enclosed in DOUBLE dollar signs ($$). For example:
   $$v = \frac{{s}}{{t}}$$
3. For inline math within text, use italics. For example: The formula is *v = s/t* where *v* is velocity.
4. **Bold** important terms using double asterisks.
5. Use *italics* for variable names in text explanations, e.g., *v* for velocity.

**Example:**
**Query:** If a car runs 20m in 2 seconds, what is its acceleration?

**Answer:**

# Acceleration Calculation for a Car

## Given Values
- Displacement (*s*) = 20 m
- Time (*t*) = 2 s
- Initial velocity (*u*) = 0 m/s (assuming the car starts from rest)
- Acceleration (*a*) = To be calculated

## Formula Selection
We'll use the equation:
$$s = ut + \frac{{1}}{{2}}at^2$$

**Reason:** This formula is appropriate because we know the displacement (*s*), time (*t*), and initial velocity (*u*). We're solving for acceleration (*a*), and this equation directly relates these quantities.

## Calculation
Starting with our chosen equation:
$$20 = (0)(2) + \frac{{1}}{{2}}a(2)^2$$

Simplifying:
$$20 = 0 + 2a$$

Solving for *a*:
$$a = \frac{{20}}{{2}} = 10 \, \text{{m/s}}^2$$

## Conclusion
The car's acceleration is **10 m/s²**.

Now use the following context items to answer the user query:
{0}
Relevant passages: <extract relevant passages from the context here>
User query: {1}
Answer:'''
    return base_prompt.format(context, query)

In [ ]:
# Helper function for text formatting
def format_text(text, length=80):
    return text

def format_text_for_chat(text):
    return text

# Shared state to control text generation
class GenerationState:
    def __init__(self):
        self.stop_generation = False

generation_state = GenerationState()

def prompt_formatter(query: str, context_items: list[dict]) -> str:
    context = ". " + "\n- ".join([page["chunk"] for page in context_items])
    base_prompt = r'''You are an assistive physics chatbot named Teammit who answers questions for 9th-grade students using their physics textbook. You can only answer physics-based questions. Follow these guidelines:

1. Carefully read the entire question and identify all parts that need to be answered.
2. Give yourself room to think by extracting relevant passages from the context before answering the query.
3. Present a clear and concise solution without unnecessary filler text.
4. Use a structured approach to problem-solving:
   a) **List given information**
   b) **State the correct formula(s)**, making sure signs are appropriately used
   c) **Explain why the chosen formula** is suitable for this problem
   d) **Outline solution steps** and substitute values correctly
   e) **Perform all necessary calculations** for each part of the question
   f) **State and explain the final answer** based on the calculations
5. Ensure you complete **all** calculations requested in the question. Double-check that you've addressed every part of the query.
6. **Never stop** at explanation; make sure you conclude who or what satisfies the query.
7. **Always base your conclusion on the calculations you performed**, even if it differs from examples in the textbook.
8. Use the formatting instructions provided (Markdown headers, LaTeX for equations, etc.)
9. Explain your reasoning clearly, making the solution process easy to follow.
10. If you cannot answer the question from the given context of the book, say: "Sorry, I cannot answer this question as it is not covered in your textbook."
11. **For math problems**, state the symbols for the terms, e.g., Distance = *s*, and then calculate using the symbols.
12. Make sure your answers are as explanatory as possible.
13. Before concluding, **review the original question** to ensure all parts have been answered. If any part is missing, complete the calculation and add it to your response.
14. For follow-up questions, first search for the context in your past conversations and then refer to the book.
15. When solving physics problems, always consider the following key equations and choose the **most appropriate one** based on the given information:
    - **For motion:**
      * $$s = ut + \frac{{1}}{{2}}at^2$$ (use when displacement, time, and either initial velocity or acceleration are known)
      * $$v = u + at$$ (use when initial velocity, time, and either final velocity or acceleration are known)
      * $$v^2 = u^2 + 2as$$ (use when initial and final velocities, and either displacement or acceleration are known)
    - **For force:** $$F = ma$$
    - **For work and energy:** $$W = Fs$$, $$KE = \frac{{1}}{{2}}mv^2$$, $$PE = mgh$$
    - **For momentum:** $$p = mv$$
    **Always explain why you chose a particular equation and how it applies to the given problem.**
16. If multiple formulas could potentially apply, **explain your reasoning** for choosing one over the others.
17. When dealing with acceleration problems, pay special attention to whether initial velocity is given or can be assumed to be zero. **Never assume a final velocity** unless it's explicitly stated or can be directly calculated from given information.
18. **Always list out all given information** and clearly state any assumptions made, explaining why those assumptions are reasonable.

**IMPORTANT FORMATTING INSTRUCTIONS:**
1. Use Markdown headers for section titles (e.g., # Main Title, ## Subtitle, ### Subsubtitle).
2. For mathematical equations displayed on their own line (block equations), use LaTeX syntax enclosed in DOUBLE dollar signs ($$). For example:
   $$v = \frac{{s}}{{t}}$$
3. For inline math within text, use italics. For example: The formula is *v = s/t* where *v* is velocity.
4. **Bold** important terms using double asterisks.
5. Use *italics* for variable names in text explanations, e.g., *v* for velocity.

**Example:**
**Query:** If a car runs 20m in 2 seconds, what is its acceleration?

**Answer:**

# Acceleration Calculation for a Car

## Given Values
- Displacement (*s*) = 20 m
- Time (*t*) = 2 s
- Initial velocity (*u*) = 0 m/s (assuming the car starts from rest)
- Acceleration (*a*) = To be calculated

## Formula Selection
We'll use the equation:
$$s = ut + \frac{{1}}{{2}}at^2$$

**Reason:** This formula is appropriate because we know the displacement (*s*), time (*t*), and initial velocity (*u*). We're solving for acceleration (*a*), and this equation directly relates these quantities.

## Calculation
Starting with our chosen equation:
$$20 = (0)(2) + \frac{{1}}{{2}}a(2)^2$$

Simplifying:
$$20 = 0 + 2a$$

Solving for *a*:
$$a = \frac{{20}}{{2}} = 10 \, \text{{m/s}}^2$$

## Conclusion
The car's acceleration is **10 m/s²**.

Now use the following context items to answer the user query:
{0}
Relevant passages: <extract relevant passages from the context here>
User query: {1}
Answer:'''
    return base_prompt.format(context, query)



#test_query = "Ovi takes 50 seconds, to travel 100m distance. If Mitu covers the same distance in 40 seconds, who goes faster ? Ovi or Mitu, definitely Mitu goes faster because she takes less time."

#format_text(test_query, length=80)

#values, indices = retrieve(query=test_query, embeddings=embeddings)

#ctx = [filtered_pages[i] for i in indices]

#prompt = prompt_formatter(query=test_query, context_items=ctx)

#format_text(prompt, length=80)


import gradio as gr
from ollama import chat
import time
import threading

# Shared state to control text generation
class GenerationState:
    def __init__(self):
        self.stop_generation = False

generation_state = GenerationState()

def generate_response(query, history):
    # Assuming these functions are defined elsewhere in your codebase
    formatted_query = format_text(query, length=80)
    values, indices = retrieve(query=query, embeddings=embeddings)
    ctx = [page_chunks[i] for i in indices]

    # Include previous messages as context
    if len(history) > 1:
        conversation_context = "Previous conversation:\n"
        for past_user_message, past_bot_message in history[:-1]:
            conversation_context += f"User: {past_user_message}\n"
            if past_bot_message:
                conversation_context += f"Bot: {past_bot_message}\n"
        conversation_context += f"Now answer this question: {query}\n"
        # Append conversation context to the query
        query = conversation_context + query

    prompt = prompt_formatter(query=query, context_items=ctx)

    formatted_prompt = format_text_for_chat(prompt)

    messages = [
        {
            'role': 'user',
            'content': formatted_prompt
        }
    ]

    try:
        response = chat('gemma2:2b-instruct-q5_K_M', messages=messages)
        return response['message']['content']
    except Exception as e:
        return f"Error: {str(e)}"

def user(user_message, history):
    return "", history + [[user_message, None]]

def bot(history):
    generation_state.stop_generation = False
    user_message = history[-1][0]
    bot_message = generate_response(user_message, history)
    history[-1][1] = ""
    for character in bot_message:
        if generation_state.stop_generation:
            break
        history[-1][1] += character
        time.sleep(0.01)  # Reduced delay for faster response
        yield history

    if generation_state.stop_generation:
        history[-1][1] += " [Generation stopped]"
        yield history

def stop_generation():
    generation_state.stop_generation = True

with gr.Blocks(theme=gr.themes.Monochrome(), css="footer{display:none !important}") as demo:
    gr.HTML("""
    <div style="text-align: center;">
        <h1>Teammit Physics Chatbot</h1>
        <p><strong>Team Members:</strong></p>
        <ul style="list-style-type: none; padding: 0; margin: 0; display: flex; justify-content: center;">
            <li style="margin: 0 15px;">Mahbub Kousar</li>
            <li style="margin: 0 15px;">Mahdi Mohammad Sifat</li>
            <li style="margin: 0 15px;">Mannan Sami</li>
        </ul>
    </div>
    """)

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type your message here...")
    with gr.Row():
        clear = gr.Button("Clear")
        stop = gr.Button("Stop")

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)
    stop.click(stop_generation, queue=False)

if __name__ == "__main__":
    setup_ollama()
    demo.launch(debug="True")

ModuleNotFoundError: No module named 'gradio'

In [ ]:
!pip install ollama

In [ ]:
import pandas as pd
from ollama import chat
import time
import logging
from pathlib import Path

# Setup logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('test_chatbot.log'),
        logging.StreamHandler()
    ]
)

def test_generate_response(query, history=[]):
    """Test version of generate_response with retrieval logic"""
    print(f"\nProcessing query: {query}")

    # Format the query
    formatted_query = format_text(query, length=80)

    # Retrieve relevant context
    values, indices = retrieve(query=query, embeddings=embeddings)
    ctx = [page_chunks[i] for i in indices]

    # Handle conversation history
    if len(history) > 1:
        conversation_context = "Previous conversation:\n"
        for past_user_message, past_bot_message in history[:-1]:
            conversation_context += f"User: {past_user_message}\n"
            if past_bot_message:
                conversation_context += f"Bot: {past_bot_message}\n"
        conversation_context += f"Now answer this question: {query}\n"
        query = conversation_context + query

    # Generate prompt
    prompt = prompt_formatter(query=query, context_items=ctx)
    formatted_prompt = format_text_for_chat(prompt)

    print(f"Generated prompt length: {len(formatted_prompt)}")
    print("Retrieved context chunks:", len(ctx))

    messages = [
        {
            'role': 'user',
            'content': formatted_prompt
        }
    ]

    try:
        print("Sending request to Ollama...")
        response = chat('gemma2:2b-instruct-q5_K_M', messages=messages)
        bot_response = response['message']['content']
        print(f"Response received (length: {len(bot_response)})")
        return bot_response
    except Exception as e:
        print(f"Error in generate_response: {str(e)}")
        return f"Error: {str(e)}"

def test_single_chapter(chapter_num=1, num_questions=3):
    """
    Test function to process a single chapter file with detailed output
    """
    file_path = f"/content/drive/MyDrive/qtr/Copy of chapter{chapter_num}_questions.csv"
    print(f"\n=== Testing Chapter {chapter_num} ===")

    try:
        # Check if file exists
        if not Path(file_path).exists():
            print(f"Error: File {file_path} not found!")
            return

        # Read the CSV file with exact column names
        print(f"Reading {file_path}...")
        df = pd.read_csv(file_path)

        # Verify required columns exist
        required_columns = ['Question', 'Answer', 'Type', 'Difficulty']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Error: Missing required columns: {missing_columns}")
            return

        print(f"Total questions in file: {len(df)}")
        print(f"Columns in CSV: {df.columns.tolist()}")

        # Create backup of original file
        backup_path = f"/content/drive/MyDrive/1. Generated Questions 1/chapter{chapter_num}_questions_backup.csv"
        df.to_csv(backup_path, index=False)
        print(f"Backup created at: {backup_path}")

        # Add Bot Output column if it doesn't exist
        if 'Bot Output' not in df.columns:
            df['Bot Output'] = None
            print("Added 'Bot Output' column")

        # Process limited number of questions for testing
        questions_to_process = df.iloc[:num_questions]
        print(f"\nProcessing {num_questions} questions...")

        # Maintain conversation history
        history = []

        for idx, row in questions_to_process.iterrows():
            print(f"\n--- Question {idx + 1} ---")
            question = row['Question']  # Note the capital 'Q'
            print(f"Question: {question}")
            print(f"Original Answer: {row['Answer']}")  # Note the capital 'A'
            print(f"Question Type: {row['Type']}")
            print(f"Difficulty: {row['Difficulty']}")

            try:
                print("\nGenerating bot response...")
                # Add current question to history
                history.append([question, None])
                bot_response = test_generate_response(question, history)
                # Update history with bot response
                history[-1][1] = bot_response

                df.loc[idx, 'Bot Output'] = bot_response
                print(f"Bot Response: {bot_response[:100]}..." if len(bot_response) > 100 else bot_response)

                # Save after each response
                df.to_csv(file_path, index=False)
                print("Response saved to CSV")

                # Add delay between questions
                print("Waiting 2 seconds before next question...")
                time.sleep(0.5) # Was 2

            except Exception as e:
                print(f"Error processing question: {str(e)}")
                continue

        print("\n=== Test Summary ===")
        print(f"Processed {num_questions} questions from Chapter {chapter_num}")
        print(f"Results saved to: {file_path}")
        print(f"Backup saved to: {backup_path}")

    except Exception as e:
        print(f"Fatal error in test_single_chapter: {str(e)}")

if __name__ == "__main__":
    print("Starting test run...")
    # Test first chapter with 3 questions
    test_single_chapter(chapter_num=2, num_questions=3)
    print("\nTest run completed!")

Starting test run...

=== Testing Chapter 2 ===
Reading /content/drive/MyDrive/qtr/Copy of chapter2_questions.csv...
Total questions in file: 32
Columns in CSV: ['Question', 'Answer', 'Type', 'Difficulty', 'Generated_At']
Backup created at: /content/drive/MyDrive/1. Generated Questions 1/chapter2_questions_backup.csv
Added 'Bot Output' column

Processing 3 questions...

--- Question 1 ---
Question: How does the concept of a reference frame help in distinguishing between rest and motion?
Original Answer: The concept of a reference frame helps in distinguishing between rest and motion by providing a fixed point or object with respect to which the position, rest, and motion of another object can be determined. This allows us to understand whether an object's position is changing or remaining the same with respect to the reference frame. By choosing a reference frame, we can accurately determine whether an object is at rest or in motion.
Question Type: Explanatory
Difficulty: Moderate

Gen

## ekhane ashbo

In [ ]:
import pandas as pd
from ollama import chat
import time
import logging
from pathlib import Path
from typing import List, Optional
from datetime import datetime

# Enhanced logging setup
def setup_logging():
    """Configure logging with both file and console handlers"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_filename = f'test_chatbot_{timestamp}.log'

    logging.basicConfig(
        level=logging.DEBUG,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

def test_generate_response(query: str, history: List = None) -> str:
    """Test version of generate_response with retrieval logic"""
    if history is None:
        history = []

    logger.debug(f"Processing query: {query}")

    # Format the query
    formatted_query = format_text(query, length=80)

    # Retrieve relevant context
    values, indices = retrieve(query=query, embeddings=embeddings)
    ctx = [page_chunks[i] for i in indices]

    # Handle conversation history
    if len(history) > 1:
        conversation_context = "Previous conversation:\n"
        for past_user_message, past_bot_message in history[:-1]:
            conversation_context += f"User: {past_user_message}\n"
            if past_bot_message:
                conversation_context += f"Bot: {past_bot_message}\n"
        conversation_context += f"Now answer this question: {query}\n"
        query = conversation_context + query

    # Generate prompt
    prompt = prompt_formatter(query=query, context_items=ctx)
    formatted_prompt = format_text_for_chat(prompt)

    logger.debug(f"Generated prompt length: {len(formatted_prompt)}")
    logger.debug(f"Retrieved context chunks: {len(ctx)}")

    messages = [
        {
            'role': 'user',
            'content': formatted_prompt
        }
    ]

    try:
        logger.debug("Sending request to Ollama...")
        response = chat('gemma2:2b-instruct-q5_K_M', messages=messages)
        bot_response = response['message']['content']
        logger.debug(f"Response received (length: {len(bot_response)})")
        return bot_response
    except Exception as e:
        logger.error(f"Error in generate_response: {str(e)}")
        return f"Error: {str(e)}"

def process_chapter(chapter_num: int, num_questions: Optional[int] = None) -> dict:
    """
    Process a single chapter and return statistics
    """
    stats = {
        'total_questions': 0,
        'processed_questions': 0,
        'errors': 0,
        'start_time': datetime.now()
    }

    file_path = f"/content/drive/MyDrive/qtr/Copy of chapter{chapter_num}_questions.csv"
    logger.info(f"\n=== Processing Chapter {chapter_num} ===")

    try:
        # Verify file exists
        if not Path(file_path).exists():
            logger.error(f"File {file_path} not found!")
            return stats

        # Read and validate CSV
        df = pd.read_csv(file_path)
        required_columns = ['Question', 'Answer', 'Type', 'Difficulty']
        missing_columns = [col for col in required_columns if col not in df.columns]

        if missing_columns:
            logger.error(f"Missing required columns: {missing_columns}")
            return stats

        stats['total_questions'] = len(df)

        # Create backup
        backup_path = f"/content/drive/MyDrive/1. Generated Questions 1/chapter{chapter_num}_questions_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(backup_path, index=False)
        logger.info(f"Backup created at: {backup_path}")

        # Add or reset Bot Output column
        df['Bot Output'] = None

        # Process questions
        questions_to_process = df.iloc[:num_questions] if num_questions else df
        history = []

        for idx, row in questions_to_process.iterrows():
            try:
                logger.info(f"\n--- Question {idx + 1}/{len(questions_to_process)} ---")
                logger.info(f"Question: {row['Question']}")
                logger.debug(f"Type: {row['Type']}, Difficulty: {row['Difficulty']}")

                history.append([row['Question'], None])
                bot_response = test_generate_response(row['Question'], history)
                history[-1][1] = bot_response

                df.loc[idx, 'Bot Output'] = bot_response
                logger.info(f"Response: {bot_response[:100]}..." if len(bot_response) > 100 else bot_response)

                # Save progress
                df.to_csv(file_path, index=False)
                stats['processed_questions'] += 1

                # Add delay between questions
                time.sleep(0.5)

            except Exception as e:
                logger.error(f"Error processing question {idx + 1}: {str(e)}")
                stats['errors'] += 1
                continue

    except Exception as e:
        logger.error(f"Fatal error processing chapter {chapter_num}: {str(e)}")
        stats['errors'] += 1

    stats['end_time'] = datetime.now()
    stats['duration'] = stats['end_time'] - stats['start_time']
    return stats

def test_all_chapters(start_chapter: int = 1, end_chapter: int = 14, questions_per_chapter: Optional[int] = None):
    """
    Test multiple chapters and generate a summary report
    """
    logger = setup_logging()
    logger.info(f"Starting full test run from Chapter {start_chapter} to {end_chapter}")

    overall_stats = {
        'total_questions_processed': 0,
        'total_errors': 0,
        'start_time': datetime.now(),
        'chapter_stats': {}
    }

    try:
        for chapter_num in range(start_chapter, end_chapter + 1):
            chapter_stats = process_chapter(chapter_num, questions_per_chapter)
            overall_stats['chapter_stats'][chapter_num] = chapter_stats
            overall_stats['total_questions_processed'] += chapter_stats['processed_questions']
            overall_stats['total_errors'] += chapter_stats['errors']

            logger.info(f"\n=== Chapter {chapter_num} Summary ===")
            logger.info(f"Questions Processed: {chapter_stats['processed_questions']}/{chapter_stats['total_questions']}")
            logger.info(f"Errors: {chapter_stats['errors']}")
            logger.info(f"Duration: {chapter_stats['duration']}")

    except Exception as e:
        logger.error(f"Fatal error in test_all_chapters: {str(e)}")

    overall_stats['end_time'] = datetime.now()
    overall_stats['total_duration'] = overall_stats['end_time'] - overall_stats['start_time']

    # Generate final report
    logger.info("\n====== Final Test Report ======")
    logger.info(f"Total Questions Processed: {overall_stats['total_questions_processed']}")
    logger.info(f"Total Errors: {overall_stats['total_errors']}")
    logger.info(f"Total Duration: {overall_stats['total_duration']}")
    logger.info(f"Average Time per Question: {overall_stats['total_duration'].total_seconds() / overall_stats['total_questions_processed'] if overall_stats['total_questions_processed'] > 0 else 0:.2f} seconds")

    return overall_stats

if __name__ == "__main__":
    logger = setup_logging()
    logger.info("Starting test run...")

    # Test all chapters with optional limit of questions per chapter
    # Set questions_per_chapter to None to process all questions
    stats = test_all_chapters(start_chapter=1, end_chapter=14, questions_per_chapter=None)

    logger.info("\nTest run completed!")

Time taken to get scores on 465 embeddings: 0.00035 seconds
Time taken to get scores on 465 embeddings: 0.00031 seconds
Time taken to get scores on 465 embeddings: 0.00029 seconds
Time taken to get scores on 465 embeddings: 0.00026 seconds
Time taken to get scores on 465 embeddings: 0.00031 seconds
Time taken to get scores on 465 embeddings: 0.00080 seconds
Time taken to get scores on 465 embeddings: 0.00029 seconds
Time taken to get scores on 465 embeddings: 0.00035 seconds
Time taken to get scores on 465 embeddings: 0.00027 seconds
Time taken to get scores on 465 embeddings: 0.00037 seconds
Time taken to get scores on 465 embeddings: 0.00024 seconds
Time taken to get scores on 465 embeddings: 0.00029 seconds
Time taken to get scores on 465 embeddings: 0.00030 seconds
Time taken to get scores on 465 embeddings: 0.00025 seconds
Time taken to get scores on 465 embeddings: 0.00029 seconds
Time taken to get scores on 465 embeddings: 0.00026 seconds
Time taken to get scores on 465 embeddin

In [ ]:
import pandas as pd
from pathlib import Path
from typing import Optional
import logging
from datetime import datetime
import torch
from sentence_transformers import util

def add_context_to_chapters(start_chapter: int = 1, end_chapter: int = 14, questions_per_chapter: Optional[int] = None):
    """
    Retrieve and save context chunks for questions in CSV files.

    Args:
        start_chapter (int): First chapter to process
        end_chapter (int): Last chapter to process
        questions_per_chapter (Optional[int]): Limit number of questions to process per chapter
    """
    logger = logging.getLogger(__name__)
    logger.info(f"Starting context retrieval for chapters {start_chapter} to {end_chapter}")

    stats = {
        'processed_files': 0,
        'total_questions': 0,
        'start_time': datetime.now()
    }

    for chapter_num in range(start_chapter, end_chapter + 1):
        file_path = f"/content/drive/MyDrive/qtr/Copy of chapter{chapter_num}_questions.csv"

        try:
            # Check if file exists
            if not Path(file_path).exists():
                logger.error(f"File {file_path} not found!")
                continue

            logger.info(f"\n=== Processing Chapter {chapter_num} ===")

            # Read CSV
            df = pd.read_csv(file_path)

            # Create backup
            backup_path = f"/content/drive/MyDrive/1. Generated Questions 1/chapter{chapter_num}_questions_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            df.to_csv(backup_path, index=False)
            logger.info(f"Backup created at: {backup_path}")

            # Add or reset Context column
            df['Context'] = None

            # Process questions
            questions_to_process = df.iloc[:questions_per_chapter] if questions_per_chapter else df

            for idx, row in questions_to_process.iterrows():
                try:
                    logger.info(f"\n--- Question {idx + 1}/{len(questions_to_process)} ---")
                    logger.info(f"Processing: {row['Question']}")

                    # Get relevant context using your retrieve function
                    _, indices = retrieve(
                        query=row['Question'],
                        embeddings=embeddings,
                        embedding_model=embedding_model,
                        retrievables=2
                    )

                    # Convert indices to CPU and numpy for processing
                    indices = indices.cpu().numpy()

                    # Get the chunks using indices and format them with page numbers
                    context_chunks = []
                    for index in indices:
                        chunk_info = page_chunks[index]
                        formatted_chunk = (
                            f"Page {chunk_info.get('page', 'Unknown')}: "
                            f"{chunk_info['chunk']}"
                        )
                        context_chunks.append(formatted_chunk)

                    # Join context chunks with separator
                    context_text = "\n---\n".join(context_chunks)

                    # Save context to dataframe
                    df.loc[idx, 'Context'] = context_text

                    # Save progress after each question
                    df.to_csv(file_path, index=False)
                    stats['total_questions'] += 1

                    # Log for verification
                    logger.debug(f"Retrieved {len(context_chunks)} chunks")
                    logger.debug(f"Context preview: {context_text[:200]}...")

                except Exception as e:
                    logger.error(f"Error processing question {idx + 1}: {str(e)}")
                    logger.exception("Detailed error:")  # This will print the full traceback
                    continue

            stats['processed_files'] += 1

        except Exception as e:
            logger.error(f"Fatal error processing chapter {chapter_num}: {str(e)}")
            logger.exception("Detailed error:")
            continue

    stats['end_time'] = datetime.now()
    stats['duration'] = stats['end_time'] - stats['start_time']

    # Generate report
    logger.info("\n====== Context Retrieval Report ======")
    logger.info(f"Files Processed: {stats['processed_files']}")
    logger.info(f"Total Questions Processed: {stats['total_questions']}")
    logger.info(f"Total Duration: {stats['duration']}")
    logger.info(f"Average Time per Question: {stats['duration'].total_seconds() / stats['total_questions'] if stats['total_questions'] > 0 else 0:.2f} seconds")

    return stats

if __name__ == "__main__":
    # Set up logging
    logger = setup_logging()
    logger.info("Starting context retrieval...")

    # Process all chapters or specify range
    # Set questions_per_chapter to None to process all questions
    stats = add_context_to_chapters(start_chapter=1, end_chapter=14, questions_per_chapter=None)

    logger.info("\nContext retrieval completed!")

Time taken to get scores on 465 embeddings: 0.00035 seconds
Time taken to get scores on 465 embeddings: 0.00029 seconds
Time taken to get scores on 465 embeddings: 0.00030 seconds
Time taken to get scores on 465 embeddings: 0.00026 seconds
Time taken to get scores on 465 embeddings: 0.00025 seconds
Time taken to get scores on 465 embeddings: 0.00026 seconds
Time taken to get scores on 465 embeddings: 0.00024 seconds
Time taken to get scores on 465 embeddings: 0.00023 seconds
Time taken to get scores on 465 embeddings: 0.00025 seconds
Time taken to get scores on 465 embeddings: 0.00023 seconds
Time taken to get scores on 465 embeddings: 0.00024 seconds
Time taken to get scores on 465 embeddings: 0.00026 seconds
Time taken to get scores on 465 embeddings: 0.00025 seconds
Time taken to get scores on 465 embeddings: 0.00026 seconds
Time taken to get scores on 465 embeddings: 0.00025 seconds
Time taken to get scores on 465 embeddings: 0.00025 seconds
Time taken to get scores on 465 embeddin